## Leer los datos

In [1]:
# Cargar librerías
import numpy as np
import pandas as pd
from sodapy import Socrata
import datetime


In [2]:
import os 
os.getcwd() 

'/home/leonardo/Documentos/ITAM'

In [3]:
#Leyendo los datos
MyAppToken = ''

client = Socrata("data.cityofnewyork.us", MyAppToken)

results = client.get("dsg6-ifza", limit = 100000)

df = pd.DataFrame.from_records(results)

In [4]:
df.shape

(50724, 34)

In [5]:
#df.head()

In [6]:
def clean_string(astr):
    '''
    Esta función reemplaza caracteres por espacios, espacios por guiones bajos, y el caracter ñ por 'ni'. 
    También quita acentos. 
    '''
    return astr.lower().replace('.', '') \
        .replace(',', '') \
        .replace(';', '') \
        .replace(':', '') \
        .replace('á', 'a') \
        .replace('é', 'e') \
        .replace('í', 'i') \
        .replace('ó', 'o') \
        .replace('ú', 'u') \
        .replace(' ', '_') \
        .replace('ñ', 'ni')

def clean_columns(df):
    '''
    Esta función corre la función clean_string pero para los nombres de las columnas. 
    '''
    for series in df:
        df.rename(columns={series:clean_string(series)}, inplace=True)
    
#Limpiando la base

print("\t-> Limpando los nombres de las columnas")
clean_columns(df)

print("\t-> Reemplazando espacios en blanco")
for col in df.select_dtypes('object'):
    df[col] = df[col].replace('\s+', ' ', regex=True)
    
print("\t-> Limpiando las observaciones")
for col in df.select_dtypes('object'):
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace('á', 'a')
        df[col] = df[col].str.replace('é', 'e')
        df[col] = df[col].str.replace('í', 'i')
        df[col] = df[col].str.replace('ó', 'o')
        df[col] = df[col].str.replace('ú', 'u')
        df[col] = df[col].str.replace(' ', '_')

print("\t-> Cambiando NA por np.nan")
for col in df.select_dtypes('object'):
    df.loc[df[col] == 'na', col] = np.nan

print("\t-> Eliminando duplicados")
df = df.drop_duplicates()
df.shape

	-> Limpando los nombres de las columnas
	-> Reemplazando espacios en blanco
	-> Limpiando las observaciones
	-> Cambiando NA por np.nan
	-> Eliminando duplicados


(38519, 34)

In [7]:
#df.head()

Checar los tipos de variables

In [8]:
df.dtypes

centername                               object
legalname                                object
building                                 object
street                                   object
borough                                  object
zipcode                                  object
phone                                    object
permitexp                                object
status                                   object
agerange                                 object
maximumcapacity                          object
dc_id                                    object
programtype                              object
facilitytype                             object
childcaretype                            object
bin                                      object
violationratepercent                     object
totaleducationalworkers                  object
averagetotaleducationalworkers           object
publichealthhazardviolationrate          object
criticalviolationrate                   

Dar formato adecuado a ciertas variables

In [9]:
df['violationavgratepercent'] = df['violationavgratepercent'].astype('float64')
df['averagepublichealthhazardiolationrate'] = df['averagepublichealthhazardiolationrate'].astype('float64')
df['avgcriticalviolationrate'] = df['avgcriticalviolationrate'].astype('float64')

Transformar a tipo fecha la variable inspection date 

In [10]:
df['inspectiondate'] = pd.to_datetime(df.inspectiondate, infer_datetime_format=False)

Crear variables para año, mes y día

In [11]:
df['inspection_year'] = df['inspectiondate'].dt.year
df['inspection_month'] = df['inspectiondate'].dt.month
df['inspection_day'] = df['inspectiondate'].dt.day
df['inspection_day_name'] = df['inspectiondate'].dt.day_name()

Crear variable llena de unos para hacer conteos

In [12]:
df['unos'] = 1

Checar missings por variable

In [13]:
df.isnull().sum()

centername                                   0
legalname                                    0
building                                    11
street                                       0
borough                                      0
zipcode                                     50
phone                                       46
permitexp                                    0
status                                       0
agerange                                  1781
maximumcapacity                              0
dc_id                                        0
programtype                                  0
facilitytype                                 0
childcaretype                                0
bin                                          0
violationratepercent                        86
totaleducationalworkers                      0
averagetotaleducationalworkers               0
publichealthhazardviolationrate             86
criticalviolationrate                       86
inspectiondat

## Análisis Exploratorio (EDA)

### Conteos y promedios de todas las inspecciones

Número de inspecciones por año

In [14]:
# help(pd.pivot_table)
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    aggfunc=np.sum, fill_value=0)
table

,unos
inspection_year,
2017.0,6914
2018.0,14580
2019.0,13257
2020.0,3711


Número de inspecciones por año y por borough

In [15]:
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)
table

borough,bronx,brooklyn,manhattan,queens,staten_island
inspection_year,,,,,
2017.0,1042,2576,1349,1673,274
2018.0,3088,4251,3386,3225,630
2019.0,2910,4130,2627,3071,519
2020.0,998,1063,759,780,111


Promedio de inspecciones por mes y por borough para cada año. Nota, 2017 tiene información a partir del quinto mes y 2020 tiene información hasta marzo

In [16]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(df, values='unos', index=['inspection_year','inspection_month'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,130.250000,322.000000,168.625000,209.125000,34.25
2018.0,257.333333,354.250000,282.166667,268.750000,52.50
2019.0,242.500000,344.166667,218.916667,255.916667,43.25
2020.0,249.500000,265.750000,189.750000,195.000000,27.75


Promedio de inspecciones por día y por borough para cada año. 

In [17]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(df, values='unos', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,6.855263,16.947368,8.875000,11.006579,1.802632
2018.0,12.502024,17.210526,13.708502,13.056680,2.550607
2019.0,11.456693,16.259843,10.342520,12.090551,2.043307
2020.0,18.481481,19.685185,14.055556,14.444444,2.055556


calculemos el promedio día de inspecciones por día para cada año sin considerar el borrough. Tendremos que obtener resultados similares que si sumamos el promedio por borough

In [18]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(df, values='unos', index=['inspection_year','inspection_month','inspection_day'],
                    aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'unos':['mean']})

,unos
,mean
inspection_year,
2017.0,45.486842
2018.0,59.028340
2019.0,52.192913
2020.0,68.722222


Número de inspecciones por año y por categoría de violación.

In [19]:
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    columns=['violationcategory'], aggfunc=np.sum, fill_value=0)
table

violationcategory,critical,general,public_health_hazard
inspection_year,,,
2017.0,1973,1846,828
2018.0,2841,5019,1174
2019.0,2399,3994,1004
2020.0,847,1051,275


Promedio de inspecciones por día y por categoría de violación para cada año. Vemos que public hazard son los que menor promedio  tienen

In [20]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(df, values='unos', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['violationcategory'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'critical':['mean'],'general':['mean'],'public_health_hazard':['mean']})

,critical,general,public_health_hazard
,mean,mean,mean
inspection_year,,,
2017.0,12.980263,12.144737,5.447368
2018.0,11.837500,20.912500,4.891667
2019.0,9.831967,16.368852,4.114754
2020.0,16.607843,20.607843,5.392157


Los siguientes son conteos de inspecciones para ciertas variables, sólo para tenerlos presente

In [21]:
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    columns=['programtype'], aggfunc=np.sum, fill_value=0)
table

programtype,all_age_camp,infant_toddler,preschool,preschool_camp,school_age_camp
inspection_year,,,,,
2017.0,360,1042,5507,0,5
2018.0,380,2190,12001,0,9
2019.0,408,2050,10765,4,30
2020.0,0,643,3068,0,0


In [22]:
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    columns=['facilitytype'], aggfunc=np.sum, fill_value=0)
table

facilitytype,camp,gdc,sbcc
inspection_year,,,
2017.0,365,5904,645
2018.0,389,12674,1517
2019.0,442,11320,1495
2020.0,0,3449,262


In [23]:
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    columns=['inspection_day_name'], aggfunc=np.sum, fill_value=0)
table

inspection_day_name,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
inspection_year,,,,,,,
2017.0,769,686,0,0,1906,1587,1966
2018.0,1500,879,6,0,4112,3627,4456
2019.0,1656,1034,0,1,3612,2980,3974
2020.0,437,230,0,0,1053,802,1189


In [24]:
table = pd.pivot_table(df, values='unos', index=['inspection_year'],
                    columns=['violationstatus'], aggfunc=np.sum, fill_value=0)
table

violationstatus,corrected,n/a,open
inspection_year,,,
2017.0,4647,2267,0
2018.0,9017,5546,17
2019.0,6884,5860,513
2020.0,1333,1538,840


### Conteos y promedios de todas las initial anual inspections

Ejecutamos el código que necesitamos para la tabla de initial anual inspections

In [25]:
# código de featurin engineering

In [26]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns #Control figure 
import numpy as np
import os
from datetime import date
matplotlib.style.use('ggplot')
%matplotlib inline
from sodapy import Socrata


In [27]:

tabla_4 = df.loc[:, ['dc_id', 'inspectiondate', 'regulationsummary', 'violationcategory', 'healthcodesubsection', 
                     'violationstatus', 'inspectionsummaryresult', 'borough']]

print("\t-> Reagrupar en tres variables Inspection Summary Result: reason, result_1 y result_2")

tabla_4['inspectionsummaryresult'] = tabla_4['inspectionsummaryresult'].astype('str')

df_2 = pd.DataFrame(tabla_4.inspectionsummaryresult.str.split('_-_',1).tolist(), columns= ['reason', 'result'])

df_2['result'] = df_2['result'].astype('str')

df_3 = pd.DataFrame(df_2.result.str.split(';_',1).tolist(), columns = ['result_1', 'result_2'])

df_2 = df_2.drop(df_2.columns[[1]], axis=1) 

df_2 = df_2.join(df_3)

tabla_4 = tabla_4.join(df_2)

tabla_4 = tabla_4.drop(['inspectionsummaryresult'], axis = 1) #Eliminar inspection_summary_result

print("\t-> Únicamente nos quedamos con initial_annual_inspection")

tabla_4.reason.value_counts(dropna=False)

tabla_4 = tabla_4.loc[tabla_4['reason'] == 'initial_annual_inspection']

print("\t-> Creamos categorias para las variables result_1 y result_2")

categorias = ["result_1", "result_2"]

df_4 = pd.get_dummies(tabla_4[categorias])

tabla_4 = tabla_4.join(df_4)

tabla_4 = tabla_4.drop(['result_1', 'result_2'], axis = 1) #Eliminamos variables que no necesitamos

print("\t-> Creamos variables de año, mes y día a partir de Inspection date")

tabla_4['inspectiondate'] = tabla_4['inspectiondate'].astype('str')

tabla_4['inspectiondate'] = pd.to_datetime(tabla_4.inspectiondate, infer_datetime_format=False)

tabla_4['inspection_year'] = tabla_4['inspectiondate'].dt.year

tabla_4['inspection_month'] = tabla_4['inspectiondate'].dt.month

tabla_4['inspection_day_name'] = tabla_4['inspectiondate'].dt.day_name()

print("\t-> Eliminamos los días no hábiles: sábado y domingo")

tabla_4 = tabla_4.drop(tabla_4.loc[tabla_4['inspection_day_name']== 'Saturday'].index)

tabla_4 = tabla_4.drop(tabla_4.loc[tabla_4['inspection_day_name']== 'Sunday'].index)

dias = {"Monday":'1', 
                    "Tuesday":'2', 
                    "Wednesday":'3', 
                    "Thursday":'4',
                    "Friday":'5'}

tabla_4['inspection_day_name'] = tabla_4['inspection_day_name'].map(dias)

tabla_4['inspection_day_name'] = tabla_4['inspection_day_name'].astype('float')

print("\t-> Renombramos la variabla dc_id por center_id")

tabla_4.rename(columns={'dc_id':'center_id'}, inplace=True)

print("\t-> Ordenamos la base por year, month y day en forma descendente")

tabla_4.sort_values(['inspectiondate'], ascending=[False], inplace=True)

print("\t-> One-hot encoding de la variable violationcategory")

categorias = ["violationcategory"]

df_5 = pd.get_dummies(tabla_4[categorias])

tabla_4 = tabla_4.join(df_5)

tabla_4 = tabla_4.drop(['violationcategory'], axis = 1) #Eliminamos variables que no necesitamos

print("\t-> Se crea la variable: ultima_inspección, que son los días que han pasado desde la última inspección anual")

df_6 = tabla_4.loc[tabla_4['inspection_year']!=2020.0]

df_7 = pd.DataFrame(df_6.groupby(["center_id"], sort=False)["inspectiondate"].max().reset_index())

year = str(pd.datetime.now().year)
month = str(pd.datetime.now().month)
day = str(pd.datetime.now().day)

fechas = year + "-" + month + "-" + day

df_7["today"] = pd.to_datetime(fechas)

df_7['dias_ultima_inspeccion'] = df_7['today'] - df_7['inspectiondate']

df_7['dias_ultima_inspeccion'] = df_7['dias_ultima_inspeccion'].dt.days

tabla_4 = pd.merge(tabla_4, df_7, left_on='center_id', right_on='center_id', how='left')

tabla_4 =  tabla_4.rename(columns = {'inspectiondate_x':'inspectiondate'})

tabla_4 = tabla_4.drop(['today', 'inspectiondate_y'], axis = 1)

print("\t-> Creamos la variable violaciones_hist_salud_publica: Número de violaciones de salud pública históricas (2017-2019) por centro")

df_8 = pd.DataFrame(df_6.groupby(["center_id"], sort=False)["violationcategory_public_health_hazard"].sum().reset_index())

df_8 =  df_8.rename(columns = {'violationcategory_public_health_hazard':'violaciones_hist_salud_publica'})

tabla_4 = pd.merge(tabla_4, df_8, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable violaciones_2019_salud_publica: Número de violaciones de salud pública en el 2019 por centro")

df_9 = tabla_4.loc[tabla_4['inspection_year']==2019.0]

df_10 = pd.DataFrame(df_9.groupby(["center_id"], sort=False)["violationcategory_public_health_hazard"].sum().reset_index())

df_10 =  df_10.rename(columns = {'violationcategory_public_health_hazard':'violaciones_2019_salud_publica'})

tabla_4 = pd.merge(tabla_4, df_10, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable violaciones_hist_criticas: Número de violaciones críticas históricas anteriores (2016-2019) por centro")

df_11 = pd.DataFrame(df_6.groupby(["center_id"], sort=False)["violationcategory_critical"].sum().reset_index())

df_11 =  df_11.rename(columns = {'violationcategory_critical':'violaciones_hist_criticas'})

tabla_4 = pd.merge(tabla_4, df_11, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable violaciones_2019_criticas: Número de violaciones críticas en el 2019 por centro")

df_12 = pd.DataFrame(df_9.groupby(["center_id"], sort=False)["violationcategory_critical"].sum().reset_index())

df_12 =  df_12.rename(columns = {'violationcategory_critical':'violaciones_2019_criticas'})

tabla_4 = pd.merge(tabla_4, df_12, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable ratio_violaciones_hist: Número de inspecciones en total de primera vez que resultaron en violación crítica o de salud pública/ número de inspecciones de primera vez por centro")

df_13 = pd.merge(df_8, df_11)

df_13['total'] = df_13['violaciones_hist_salud_publica'] + df_13['violaciones_hist_criticas']

df_14 = pd.DataFrame(df_6.groupby(["center_id"], sort=False)["reason"].count().reset_index())

df_15 = pd.merge(df_13, df_14)

df_15['ratio_violaciones_hist'] = df_15['total'] / df_15['reason']

df_15 = df_15.drop(['violaciones_hist_salud_publica', 'violaciones_hist_criticas', 'total', 'reason'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_15, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable ratio_violaciones_2019: Número de inspecciones en total de primera vez que resultaron en violación crítica o de salud pública en el 2019 / número de inspecciones de primera vez por centro")

df_16 = pd.merge(df_10, df_12)

df_16['total'] = df_16['violaciones_2019_salud_publica'] + df_16['violaciones_2019_criticas']

df_17 = pd.DataFrame(df_9.groupby(["center_id"], sort=False)["reason"].count().reset_index())

df_18 = pd.merge(df_16, df_17)

df_18['ratio_violaciones_2019'] = df_18['total'] / df_18['reason']

df_18 = df_18.drop(['violaciones_2019_salud_publica', 'violaciones_2019_criticas', 'total', 'reason'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_18, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable prom_violaciones_hist_borough: Promedio de violaciones históricas por distrito")

df_19 = pd.DataFrame(df_6.groupby(["borough"], sort=False)[["violationcategory_critical", "violationcategory_general", "violationcategory_public_health_hazard"]].sum().reset_index())

df_19['prom_violaciones_hist_borough'] = df_19[['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard']].mean(axis=1)

df_19 = df_19.drop(['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_19, left_on='borough', right_on='borough', how='left')

print("\t-> Creamos la variable prom_violaciones_2019_borough: Promedio de violaciones en el 2019 por distrito")

df_20 = pd.DataFrame(df_9.groupby(["borough"], sort=False)[["violationcategory_critical", "violationcategory_general", "violationcategory_public_health_hazard"]].sum().reset_index())

df_20['prom_violaciones_2019_borough'] = df_20[['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard']].mean(axis=1)

df_20 = df_20.drop(['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_20, left_on='borough', right_on='borough', how='left')

print("\t-> Creamos la variable ratio_violaciones_hist_sp: Número de violaciones de salud pública de primera vez por centro históricas (2017-2019)/ número de violaciones de primera vez de todo tipo por centro históricas (2017-2019) ")

df_21 = pd.DataFrame(df_6.groupby(["center_id"], sort=False)[["violationcategory_critical", "violationcategory_general", "violationcategory_public_health_hazard"]].sum().reset_index())

df_21['total'] = df_21['violationcategory_critical'] + df_21['violationcategory_general'] + df_21['violationcategory_public_health_hazard']

df_21['ratio_violaciones_hist_sp'] = df_21['violationcategory_public_health_hazard'] / df_21['total']

df_21 = df_21.drop(['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard', 'total'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_21, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable ratio_violaciones_2019_sp: Número de violaciones de salud pública de primera vez por centro en el 2019 / número de violaciones de primera vez de todo tipo por centro en el 2019 ")

df_22 = pd.DataFrame(df_9.groupby(["center_id"], sort=False)[["violationcategory_critical", "violationcategory_general", "violationcategory_public_health_hazard"]].sum().reset_index())

df_22['total'] = df_22['violationcategory_critical'] + df_22['violationcategory_general'] + df_22['violationcategory_public_health_hazard']

df_22['ratio_violaciones_2019_sp'] = df_22['violationcategory_public_health_hazard'] / df_22['total']

df_22 = df_22.drop(['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard', 'total'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_22, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable ratio_violaciones_hist_criticas: Número de violaciones críticas de primera vez por centro históricas (2017-2019)/ número de violaciones de primera vez de todo tipo por centro históricas (2017-2019)")

df_23 = pd.DataFrame(df_6.groupby(["center_id"], sort=False)[["violationcategory_critical", "violationcategory_general", "violationcategory_public_health_hazard"]].sum().reset_index())

df_23['total'] = df_23['violationcategory_critical'] + df_23['violationcategory_general'] + df_23['violationcategory_public_health_hazard']

df_23['ratio_violaciones_hist_criticas'] = df_23['violationcategory_critical'] / df_23['total']

df_23 = df_23.drop(['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard', 'total'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_23, left_on='center_id', right_on='center_id', how='left')

print("\t-> Creamos la variable ratio_violaciones_2019_criticas: Número de violaciones críticas de primera vez por centro en el 2019/ número de violaciones de primera vez de todo tipo por centro en el 2019")

df_24 = pd.DataFrame(df_9.groupby(["center_id"], sort=False)[["violationcategory_critical", "violationcategory_general", "violationcategory_public_health_hazard"]].sum().reset_index())

df_24['total'] = df_24['violationcategory_critical'] + df_24['violationcategory_general'] + df_24['violationcategory_public_health_hazard']

df_24['ratio_violaciones_2019_criticas'] = df_24['violationcategory_critical'] / df_24['total']

df_24 = df_24.drop(['violationcategory_critical', 'violationcategory_general', 'violationcategory_public_health_hazard', 'total'], axis = 1)

tabla_4 = pd.merge(tabla_4, df_24, left_on='center_id', right_on='center_id', how='left')

tabla_4.info()

	-> Reagrupar en tres variables Inspection Summary Result: reason, result_1 y result_2
	-> Únicamente nos quedamos con initial_annual_inspection
	-> Creamos categorias para las variables result_1 y result_2
	-> Creamos variables de año, mes y día a partir de Inspection date
	-> Eliminamos los días no hábiles: sábado y domingo
	-> Renombramos la variabla dc_id por center_id
	-> Ordenamos la base por year, month y day en forma descendente
	-> One-hot encoding de la variable violationcategory
	-> Se crea la variable: ultima_inspección, que son los días que han pasado desde la última inspección anual
	-> Creamos la variable violaciones_hist_salud_publica: Número de violaciones de salud pública históricas (2017-2019) por centro
	-> Creamos la variable violaciones_2019_salud_publica: Número de violaciones de salud pública en el 2019 por centro
	-> Creamos la variable violaciones_hist_criticas: Número de violaciones críticas históricas anteriores (2016-2019) por centro
	-> Creamos la variable

Nuestra tabla de interes es la tabla 4

In [28]:
tabla_4.shape

(18754, 34)

Creamos variables

In [29]:
tabla_4['unos'] = 1
tabla_4['inspection_day'] = tabla_4['inspectiondate'].dt.day

Número de inspecciones iniciales anuales por año

In [30]:
# help(pd.pivot_table)
table = pd.pivot_table(tabla_4, values='unos', index=['inspection_year'],
                    aggfunc=np.sum, fill_value=0)
table

,unos
inspection_year,
2017.0,3369
2018.0,7090
2019.0,6434
2020.0,1831


Número de inspecciones iniciales anuales por año y por borough

In [31]:
table = pd.pivot_table(tabla_4, values='unos', index=['inspection_year'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)
table

borough,bronx,brooklyn,manhattan,queens,staten_island
inspection_year,,,,,
2017.0,512,1256,665,807,129
2018.0,1522,2050,1669,1521,328
2019.0,1425,2025,1274,1462,248
2020.0,503,505,377,394,52


Promedio de inspecciones iniciales anuales por día y por borough para cada año. 

In [32]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='unos', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,3.368421,8.263158,4.375000,5.309211,0.848684
2018.0,6.263374,8.436214,6.868313,6.259259,1.349794
2019.0,5.722892,8.132530,5.116466,5.871486,0.995984
2020.0,9.490566,9.528302,7.113208,7.433962,0.981132


Vemos que el promedio diario de las inspecciones iniciales anuales son menores a 10 para cada borough 

### Promedios diarios de la tabla de initial inspections para las variables a las que se les hizo featuring engineering y one hot encoding

Promedio diario de inspecciones iniciales anuales de public hazard y por borough para cada año.

In [33]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='violationcategory_public_health_hazard', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,0.493421,0.967105,0.460526,0.796053,0.072368
2018.0,0.600823,0.526749,0.543210,0.625514,0.065844
2019.0,0.526104,0.493976,0.393574,0.594378,0.032129
2020.0,0.830189,0.415094,0.830189,0.811321,0.037736


Promedio diario de inspecciones iniciales anuales de critical y por borough para cada año.

In [34]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='violationcategory_critical', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,0.907895,2.256579,1.210526,1.625000,0.223684
2018.0,1.625514,1.349794,1.139918,1.399177,0.181070
2019.0,1.373494,1.240964,0.763052,1.176707,0.108434
2020.0,3.245283,1.641509,1.056604,1.716981,0.113208


Promedio diario de inspecciones iniciales anuales de general y por borough para cada año.

In [35]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='violationcategory_general', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)

# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,1.065789,2.282895,0.888158,1.328947,0.157895
2018.0,2.358025,2.843621,2.465021,1.888889,0.485597
2019.0,1.931727,2.172691,1.574297,1.562249,0.333333
2020.0,3.094340,2.811321,2.000000,1.962264,0.226415


Promedio diario de inspecciones iniciales anuales de reinspección requerida y por borough para cada año.

In [36]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='result_1_reinspection_required', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)


# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,1.986842,4.809211,2.434211,2.973684,0.513158
2018.0,3.572016,4.736626,3.909465,3.567901,0.753086
2019.0,3.228916,4.506024,2.827309,3.309237,0.554217
2020.0,5.000000,5.603774,3.924528,4.358491,0.452830


Promedio diario de inspecciones iniciales anuales de reinspección no requerida y por borough para cada año.

In [37]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='result_1_reinspection_not_required', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)


# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,0.434211,1.177632,0.578947,0.782895,0.125000
2018.0,0.909465,1.213992,1.012346,0.958848,0.201646
2019.0,0.799197,1.297189,0.674699,0.883534,0.136546
2020.0,1.415094,1.188679,0.886792,1.226415,0.169811


'

Promedio diario de inspecciones iniciales anuales de violaciones corregdas en la inspección y por borough para cada año.

In [38]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='result_2_violations_corrected_at_time_of_inspection', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)


# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,0.052632,0.118421,0.065789,0.111842,0.006579
2018.0,0.111111,0.135802,0.086420,0.082305,0.020576
2019.0,0.060241,0.104418,0.072289,0.064257,0.016064
2020.0,0.132075,0.207547,0.132075,0.094340,0.000000


Promedio diario de inspecciones iniciales anuales de programa cerrado y por borough para cada año.

In [39]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='result_2_program_closed', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)


# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,0.026316,0.019737,0.026316,0.026316,0.006579
2018.0,0.028807,0.037037,0.024691,0.028807,0.008230
2019.0,0.032129,0.044177,0.008032,0.016064,0.004016
2020.0,0.056604,0.037736,0.000000,0.018868,0.000000


Promedio diario de inspecciones iniciales anuales de multas pendientes y por borough para cada año.

In [40]:
# Agrupamos datos (tendremos un multi indice)
table = pd.pivot_table(tabla_4, values='result_2_fines_pending', index=['inspection_year','inspection_month','inspection_day'],
                    columns=['borough'], aggfunc=np.sum, fill_value=0)


# Calculamos el promedio (utimilzamos sólo el indice de año)
table.groupby(['inspection_year']).agg({'bronx':['mean'],'brooklyn':['mean'],'manhattan':['mean'],'queens':['mean'],'staten_island':['mean']})

,bronx,brooklyn,manhattan,queens,staten_island
,mean,mean,mean,mean,mean
inspection_year,,,,,
2017.0,1.407895,3.513158,1.690789,2.085526,0.394737
2018.0,2.625514,3.349794,2.851852,2.572016,0.551440
2019.0,2.409639,3.236948,2.016064,2.413655,0.385542
2020.0,3.415094,4.188679,2.886792,3.207547,0.396226


In [41]:
#def plot_totales(df):
#    save_path = '/home/leonardo/Documentos/ITAM/borough_inspecciones_totales.png'
#    dff = tabla_4.groupby(['borough']).agg({'result_1_passed_inspection':'sum', 'result_1_passed_inspection_with_no_violations':'sum', 'result_1_previously_cited_violations_corrected':'sum'}).reset_index()
#    dff_long = dff.melt(id_vars=['borough'], var_name="medicion", value_name="valor")
#    dff_long = dff_long[(dff_long.medicion == 'consumo_total_mixto') | (dff_long.medicion == 'consumo_total_dom') | (dff_long.medicion == 'consumo_total_no_dom')]
#    order = dff_long.groupby(['borough']).agg({'valor':'sum'}).reset_index().sort_values(['valor'], ascending=False).borough.values
#    p = sns.catplot(x="borough", y="valor", hue='medicion',data=dff_long, kind='bar',order=order, palette=color_dict)
#    plt.xticks(rotation=90)
#    plt.ticklabel_format(style='plain', axis='y')
#    plt.ylabel('Inspecciones por ciudad')
#    plt.xlabel('borough')
#    p.savefig(save_path,bbox_inches='tight')
#    plt.clf()

In [42]:
#plot_totales(df)

In [43]:
#def plot_totales_como_porcentajes(df):
#    save_path = 'docs/img/catplot_consumos_totales_porcentajes.png'
#    dff = df.groupby(['alcaldia']).agg({'consumo_total':'sum',
#                                    'consumo_total_dom': 'sum',
#                                    'consumo_total_no_dom': 'sum',
#                                    'consumo_total_mixto':'sum'}).reset_index()
#    dff.consumo_total = dff.consumo_total / dff.consumo_total.sum()
#    dff.consumo_total_dom = dff.consumo_total_dom / dff.consumo_total_dom.sum()
#    dff.consumo_total_no_dom = dff.consumo_total_no_dom / dff.consumo_total_no_dom.sum()
#    dff.consumo_total_mixto = dff.consumo_total_mixto / dff.consumo_total_mixto.sum()

#    dff_long = dff.drop(['consumo_total'], axis=1).melt(id_vars=['alcaldia'], var_name="medicion", value_name="valor")

#    order = dff.sort_values(['consumo_total'], ascending=False).alcaldia.values
#    p = sns.catplot(x="alcaldia", y="valor", hue='medicion',data=dff_long, kind='bar', palette=color_dict, order=order)
#    plt.xticks(rotation=90)
#    plt.ylabel('Consumos totales en porcentajes')
#    plt.xlabel('Alcaldía')
#    p.savefig(save_path,bbox_inches='tight')
#    plt.clf()